In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()
from pprint import pprint
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage

In [3]:
os.environ['AZURE_OPENAI_API_KEY'] = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_API_ENDPOINT = os.getenv('AZURE_API_ENDPOINT')
AZURE_DEPLOYMENT_NAME = os.getenv('AZURE_DEPLOYMENT_NAME')
AZURE_API_VERSION = os.getenv('AZURE_API_VERSION')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT') 

In [4]:
llm = AzureChatOpenAI(
    model='gpt-4.1-mini',
    azure_endpoint=AZURE_API_ENDPOINT,
    deployment_name=AZURE_DEPLOYMENT_NAME,
    api_version=AZURE_API_VERSION
)

In [12]:
messages = [
    AIMessage(content="please tell me how can I help", name='LLMModel'),
    HumanMessage(content="I want to learn coding", name='Kmk')
]

In [13]:
messages.append(AIMessage(content="That's great! What programming language are you interested in?", name='LLMModel'))

In [55]:
def add(a:int, b:int) -> int:
    """
    Add a and b 
    Args:
        a (int): first int 
        b (int): second int 
    Returns:
    int
    """
    return a + b

In [56]:
llm_with_tools = llm.bind_tools([add])

In [ ]:
tool_calls = llm_with_tools.invoke([HumanMessage(content='What is 2 plus 2')])

In [ ]:
tool_calls.tool_calls

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from typing import Annotated

class State(TypedDict):
    message:Annotated[list[AnyMessage], add_messages]

In [14]:
initial_message = [AIMessage(content='Please tell me how can I help')]

In [ ]:
initial_message.append(HumanMessage(content="I wan to learn coding"))
initial_message

In [ ]:
ai_message = AIMessage(content="Which progamming language you want to learn")
add_messages(initial_message, ai_message)

In [63]:
def llm_tool(State):
    return {"message": [llm_with_tools.invoke(State['message'])]}

In [64]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

In [ ]:
builder = StateGraph(State)

builder.add_node("llm_tool", llm_tool)
builder.add_edge(START, "llm_tool")
builder.add_edge("llm_tool", END)

In [66]:
graph = builder.compile()


In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [67]:
messages = graph.invoke({"message": "What is 2 plus 2"})
for message in messages["message"]:
    message.pretty_print()

In [75]:
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [70]:
tools = [add]

In [ ]:
builder = StateGraph(State)

builder.add_node('llm_tool', llm_tool)
builder.add_node('tools', ToolNode(tools))

builder.add_edge(START, 'llm_tool')

builder.add_conditional_edges(
    'llm_tool',
    tools_condition
)
builder.add_edge('tools', END)

In [ ]:
graph = builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({"message": "What is Machine learning"})
for message in messages["message"]:
    message.pretty_print()